In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
from math import pi
import os

def generate_radar_plots(file_path):
    # --- Parse filename for saving
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    output_dir = "figures"
    os.makedirs(output_dir, exist_ok=True)

    # --- Load and parse
    df = pd.read_csv(file_path, delimiter=';')
    result_columns = ['VL', 'pre_post', 'pre', 'post', 'pre_VL_post', 'pre_VL', 'VL_post']
    
    def parse_result_column(value):
        try:
            return ast.literal_eval(value)
        except Exception:
            return {}

    for col in result_columns:
        df[col] = df[col].apply(parse_result_column)

    records = []
    for _, row in df.iterrows():
        vuln_type = row["vulnerability_type"]
        for strategy in result_columns:
            data = row[strategy]
            if isinstance(data, dict):
                records.append({
                    "Vulnerability_Type": vuln_type,
                    "Strategy": strategy,
                    "Sanity_Test_Success": int(data.get("Sanity_Test_Success", False)),
                    "Exploit_Covered": int(data.get("Exploit_Covered", False)),
                    "Accepted_Patch": int(data.get("Sanity_Test_Success", False) and data.get("Exploit_Covered", False))
                })

    long_df = pd.DataFrame(records)
    grouped = long_df.groupby(["Vulnerability_Type", "Strategy"]).sum().reset_index()

    radar1_grouped = grouped.groupby("Vulnerability_Type")[["Sanity_Test_Success", "Exploit_Covered", "Accepted_Patch"]].sum().reset_index()
    radar2_grouped = grouped.groupby("Strategy")[["Sanity_Test_Success", "Exploit_Covered", "Accepted_Patch"]].sum().reset_index()

    best_strategy_per_vuln = (
        grouped.loc[grouped.groupby("Vulnerability_Type")["Accepted_Patch"].idxmax()]
        [["Vulnerability_Type", "Strategy", "Accepted_Patch"]]
    )

    radar1_enriched = radar1_grouped.merge(best_strategy_per_vuln, on="Vulnerability_Type")

    # --- Plotting functions
    def plot_radar_multi_clean(ax, labels, data, title, colors):
        N = len(labels)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]
        for i, metric in enumerate(data.columns[1:]):
            values = data[metric].tolist()
            values += values[:1]
            ax.plot(angles, values, linewidth=2, linestyle='solid', label=metric, color=colors[i])
            ax.fill(angles, values, alpha=0.1, color=colors[i])
            for j, val in enumerate(values[:-1]):
                ax.text(angles[j], val + 2, str(val), ha='center', va='bottom', fontsize=10, color='black', fontweight='bold')
        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, fontsize=10)
        ax.set_yticklabels([])
        ax.set_yticks([])
        ax.set_title(title, y=1.1, fontsize=14)
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=9)
        ax.spines["polar"].set_visible(False)

    def plot_radar_with_best_strategy_offset(ax, labels, data, title, colors):
        N = len(labels)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]
        for i, metric in enumerate(data.columns[1:4]):
            values = data[metric].tolist()
            values += values[:1]
            ax.plot(angles, values, linewidth=2, linestyle='solid', label=metric, color=colors[i])
            ax.fill(angles, values, alpha=0.1, color=colors[i])
            for j, val in enumerate(values[:-1]):
                ax.text(angles[j], val + 2, str(val), ha='center', va='bottom', fontsize=10, color='black', fontweight='normal')
        max_radius = max(data[["Sanity_Test_Success", "Exploit_Covered", "Accepted_Patch_x"]].max())
        custom_offsets = [23, 24, 43, 32, 23, 23, 31, -18, 28]  # May need generalization
        for i, label in enumerate(labels):
            strategy = data.iloc[i]["Strategy"]
            count = data.iloc[i]["Accepted_Patch_y"]
            angle = angles[i]
            offset = custom_offsets[i % len(custom_offsets)] + max_radius
            ax.text(angle, offset, f'{strategy}\n({count})', ha='center', va='center', fontsize=9, color='black', fontweight='bold')
        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, fontsize=10)
        ax.set_yticklabels([])
        ax.set_yticks([])
        ax.set_title(title, y=1.1, fontsize=14)
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=9)
        ax.spines["polar"].set_visible(False)

    # --- Plot and save
    fig1, ax1 = plt.subplots(subplot_kw=dict(polar=True), figsize=(8, 7))
    plot_radar_with_best_strategy_offset(
        ax1,
        radar1_enriched["Vulnerability_Type"],
        radar1_enriched,
        f"Best Strategy per Vulnerability Type ({base_filename})",
        ['#a6d854', '#66c2a5', '#8da0cb']
    )
    fig1.tight_layout()
    fig1_path = os.path.join(output_dir, f"{base_filename}_radar_best_strategy.pdf")
    fig1.savefig(fig1_path, format="pdf")
    plt.close(fig1)

    fig2, ax2 = plt.subplots(subplot_kw=dict(polar=True), figsize=(8, 7))
    plot_radar_multi_clean(
        ax2,
        radar2_grouped["Strategy"],
        radar2_grouped,
        f"Patch Performance by Strategy ({base_filename})",
        ['#a6d854', '#66c2a5', '#8da0cb']
    )
    fig2.tight_layout()
    fig2_path = os.path.join(output_dir, f"{base_filename}_radar_strategy_performance.pdf")
    fig2.savefig(fig2_path, format="pdf")
    plt.close(fig2)

    return fig1_path, fig2_path

In [ ]:
def plot_figures(list_of_csv_files):
    fig_paths = []
    for file_path in list_of_csv_files:
        fig1_path, fig2_path = generate_radar_plots(file_path)
        fig_paths.append((fig1_path, fig2_path))
        print("Saved plots:", fig1_path, fig2_path)
    return fig_paths


In [4]:
fig1_path, fig2_path = generate_radar_plots("data/validation_results_CL_fixed.csv")
print("Saved plots:", fig1_path, fig2_path)


Saved plots: figures/validation_results_CL_fixed_radar_best_strategy.pdf figures/validation_results_CL_fixed_radar_strategy_performance.pdf


In [5]:
fig1_path, fig2_path = generate_radar_plots("data/validation_results_20K_fixed.csv")
print("Saved plots:", fig1_path, fig2_path)

Saved plots: figures/validation_results_20K_fixed_radar_best_strategy.pdf figures/validation_results_20K_fixed_radar_strategy_performance.pdf


In [6]:
fig1_path, fig2_path = generate_radar_plots("data/validation_results_100K_fixed.csv")
print("Saved plots:", fig1_path, fig2_path)

Saved plots: figures/validation_results_100K_fixed_radar_best_strategy.pdf figures/validation_results_100K_fixed_radar_strategy_performance.pdf
